In [23]:
import os
import pandas as pd
import matplotlib as mlt
from pathlib import Path

In [24]:
data_load = Path ("../Resources/wine_data.csv")
wine_df = pd.read_csv(data_load,encoding="ISO-8859-1")
wine_df = wine_df.drop(["Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10",	"Unnamed: 11",	"Unnamed: 12"], axis=1)
wine_df.head()

C:\Users\francesca\AppData\Local\Temp\ipykernel_1868\334466515.py:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  wine_df = pd.read_csv(data_load,encoding="ISO-8859-1")


,country,description,points,price,region,title,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Etna,Nicosia 2013 Vulkà Bianco (Etna),White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
